Tools, helpful functions for development and debugging
====================



In [2]:
from sage.algebras.flag_algebras import *

In [7]:
G = GraphTheory
A = FlagAlgebra(QQ, G)
G.exclude(G(3))
#G.optimize(G(2), 3)
l = G.generate(4)

Flag Algebra Element over Rational Field
1 - Flag on 4 points, ftype from [] with edges=[[0, 2], [1, 3]]
1 - Flag on 4 points, ftype from [] with edges=[[0, 2], [0, 3], [1, 3]]
1 - Flag on 4 points, ftype from [] with edges=[[0, 2], [0, 3], [2, 3]]
1 - Flag on 4 points, ftype from [] with edges=[[0, 2], [0, 3], [1, 3], [2, 3]]
1 - Flag on 4 points, ftype from [] with edges=[[0, 2], [0, 3], [1, 2], [1, 3]]
1 - Flag on 4 points, ftype from [] with edges=[[0, 2], [0, 3], [1, 2], [1, 3], [2, 3]]
1 - Flag on 4 points, ftype from [] with edges=[[0, 1], [0, 2], [0, 3], [1, 2], [1, 3], [2, 3]]

In [ ]:
import itertools
from sage.rings.rational_field import QQ
from cysignals.signals cimport sig_check
from sage.structure.element cimport Element

cpdef _subblock_helper(list points, list block):
    r"""
    Helper to find induced substructures
    
    TESTS::

        sage: _subblock_helper([0, 1, 2], [[0, 1, 2], [2, 4], [1, 1, 2]])
        [[0, 1, 2], [1, 1, 2]]
    """
    cdef bint gd = 0
    ret = []
    if len(block)==0:
        return ret
    for xx in block:
        gd = 1
        for yy in xx:
            if yy not in points:
                gd = 0
                break
        if gd:
            ret.append([points.index(ii) for ii in xx])
    return ret

cdef class Flag(Element):
    
    cdef int _n
    cdef int _ftype_size
    
    cdef list _ftype_points
    cdef list _not_ftype_points
    cdef dict _blocks
    cdef dict _blocks_optional
    cdef bint _pure_flag
    cdef tuple _unique
    
    cdef Flag _ftype
    
    def __init__(self, theory, n, **params):
        r"""
        Initialize a :class:`Flag` element

        INPUT:

        - ``theory`` -- :class:`CombinatorialTheory`; the underlying theory, 
            which is also the parent
        - ``n`` -- integer; the size (number of vertices) of the flag
        - ``**params`` -- optional parameters; can contain the points of the
            ftype as a list of points under the name "ftype" of "ftype_points", 
            if not provided then empty ftype is assumed. Can also contain the 
            blocks for each element in the signature, if not provided then an 
            empty block set is assumed.

        OUTPUT: The resulting flag

        EXAMPLES::

        Create a simple GraphTheory triangle ::
            
            sage: from sage.algebras.flag_algebras import *
            sage: from sage.algebras.flag import Flag
            sage: Flag(GraphTheory, 3, edges=[[0, 1], [0, 2], [1, 2]])
            Flag on 3 points, ftype from [] with edges=[[0, 1], [0, 2], [1, 2]]
        
        Create a DiGraphTheory edge out from a pointed ftype ::

            sage: Flag(DiGraphTheory, 2, edges=[[0, 1]], ftype=[0])
            Flag on 2 points, ftype from [0] with edges=[[0, 1]]
        
        .. NOTE::

            It is recommended to create flags using the parent's element constructor

        .. SEEALSO::

            :func:`CombinatorialTheory._element_constructor_`
        """
        self._n = int(n)
        
        if 'ftype_points' in params:
            ftype_points = params['ftype_points']
        elif 'ftype' in params:
            ftype_points = params['ftype']
        else:
            ftype_points = []
        
        self._ftype_size = len(ftype_points)
        self._ftype_points = list(ftype_points)
        self._not_ftype_points = None
        self._pure_flag = True
        self._blocks = {}
        self._blocks_optional = {}
        for xx in theory._signature.keys():
            self._blocks[xx] = []
            if xx in params:
                self._blocks[xx] = [list(yy) for yy in params[xx]]
            self._blocks_optional[xx] = []
            for xx_opti in [xx+"_o", xx+"_optional", xx+"_opti", "o_"+xx, "optional_"+xx, "opti_"+xx]:
                if xx_opti in params:
                    xx_oblocks = [list(yy) for yy in params[xx_opti]]
                    for ed in xx_oblocks:
                        if len(_subblock_helper(self._ftype_points, xx_oblocks))!=0:
                            raise ValueError("Can't have optional blocks in ftype")
                    self._blocks_optional[xx] = xx_oblocks
                    self._pure_flag = False
                    break
        self._unique = ()
        self._ftype = None
        Element.__init__(self, theory)
    
    def _repr_(self):
        #TODO
        r"""
        Return a nice representation.
        
        If it is an ftype (all the points are part of the ftype), then
        the text changes to indicate this fact.

        EXAMPLES::

        For flags ::

            sage: from sage.algebras.flag_algebras import *
            sage: GraphTheory(3) 
            Flag on 3 points, ftype from [] with edges=[]
            
        For ftypes ::
        
            sage: ThreeGraphTheory(4, ftype_points=[0, 1, 2, 3], edges=[[0, 1, 2]])
            Ftype on 4 points with edges=[[0, 1, 2]]
        """
        blocks = self.blocks()
        strblocks = ', '.join([xx+'='+str(blocks[xx]) for xx in blocks.keys()])
        if self.is_ftype():
            return 'Ftype on {} points with {}'.format(self.size(), strblocks)
        return 'Flag on {} points, ftype from {} with {}'.format(self.size(), self.ftype_points(), strblocks)
    
    def compact_repr(self):
        #TODO
        blocks = self.blocks()
        ret = ["n:{}".format(self.size())]
        if len(self._ftype_points)!=0:
            ret.append("t:"+"".join(map(str, self._ftype_points)))
        for name in self.theory()._signature.keys():
            desc = name + ":"
            arity = self.theory()._signature[name]
            if arity==1:
                desc += "".join([str(xx[0]) for xx in blocks[name]])
            else:
                desc += ",".join(["".join(map(str, ed)) for ed in blocks[name]])
            ret.append(desc)
        return "; ".join(ret)
            
    
    def raw_numbers(self):
        #TODO
        r"""
        Return a list of numbers uniquely describing this flag.
        
        This is used in saving and loading calculations

        EXAMPLES::

            sage: from sage.algebras.flag_algebras import *
            sage: GraphTheory(2, edges=[[0, 1]], ftype=[0]).raw_numbers()
            [2, 0, 15, 0, 1, 15]
            
        """
        numbers = [self.size()] + self.ftype_points() + [15]
        blocks = self.blocks()
        for xx in blocks:
            for yy in blocks[xx]:
                numbers += yy
            numbers.append(15)
        return numbers
    
    cpdef subflag(self, points=None, ftype_points=None):
        r"""
        Returns the induced subflag.
        
        The resulting sublaf contains the union of points and ftype_points
        and has ftype constructed from ftype_points. 

        INPUT:

        - ``points`` -- list (default: `None`); the points inducing the subflag.
            If not provided (or `None`) then this is the entire vertex set, so
            only the ftype changes
        - ``ftype_points`` list (default: `None`); the points inducing the ftype
            of the subflag. If not provided (or `None`) then the original ftype
            point set is used, so the result of the ftype will be the same

        OUTPUT: The induced sub Flag

        EXAMPLES::

        Same ftype ::

            sage: from sage.algebras.flag_algebras import *
            sage: g = GraphTheory(3, edges=[[0, 1]], ftype=[0])
            sage: g.subflag([0, 2])
            Flag on 2 points, ftype from [0] with edges=[]
            
        Only change ftype ::
            
            sage: g.subflag(ftype_points=[0, 1])
            Flag on 3 points, ftype from [0, 1] with edges=[[0, 1]]

        .. NOTE::

            As the ftype points can be chosen, the result can have different
            ftype as self.

        TESTS::

            sage: g.subflag()==g
            True
        """
        if ftype_points==None:
            ftype_points = self._ftype_points
        if points==None:
            points = list(range(self._n))
        else:
            points = [ii for ii in range(self._n) if (ii in points or ii in ftype_points)]
        if len(points)==self._n and ftype_points==self._ftype_points:
            return self
        if not self._pure_flag and ftype_points!=self._ftype_points:
            raise ValueError("Subflag for patterns is not defined with different ftype!")
        blocks = {xx: _subblock_helper(points, self._blocks[xx]) for xx in self._blocks.keys()}
        if not self._pure_flag:
            blocks_optional = {xx: _subblock_helper(points, self._blocks_optional[xx]) for xx in self._blocks_optional.keys()}
        else:
            blocks_optional = {}
        new_ftype_points = [points.index(ii) for ii in ftype_points]
        return self.__class__(self.parent(), len(points), ftype=new_ftype_points, **(blocks|blocks_optional))
    
    def combinatorial_theory(self):
        r"""
        Returns the combinatorial theory this flag is a member of
        
        This is the same as the parent.

        .. SEEALSO::

            :func:`theory`
            :func:`parent`
        """
        return self.parent()
    
    theory = combinatorial_theory
    
    def as_flag_algebra_element(self, basis=QQ):
        r"""
        Transforms this `Flag` to a `FlagAlgebraElement` over a given basis

        INPUT:

        - ``basis`` -- Ring (default: `QQ`); the base of
            the FlagAlgebra where the target will live.

        OUTPUT: A `FlagAlgebraElement` representing this `Flag`

        .. SEEALSO::

            :class:`FlagAlgebra`
            :func:`FlagAlgebra._element_constructor_`
            :class:`FlagAlgebraElement`
        """
        from sage.algebras.flag_algebras import FlagAlgebra
        targ_alg = FlagAlgebra(basis, self.theory(), self.ftype())
        if self._pure_flag:
            return targ_alg(self)
        else:
            return sum(self.compatible_flags())
    
    afae = as_flag_algebra_element
    
    def as_operand(self):
        r"""
        Turns this `Flag` into a `FlagAlgebraElement` so operations can be performed on it

        .. SEEALSO::

            :func:`as_flag_algebra_element`
        """
        return self.afae(QQ)
    
    cpdef is_compatible(self, other):
        if self._n > other._n:
            return False
        if self.ftype() != other.ftype():
            return False
        for perm in itertools.permutations(other.not_ftype_points(), len(self.not_ftype_points())):
            
            operm = other.subflag()
    
    cpdef size(self):
        r"""
        Returns the size of the vertex set of this Flag.

        OUTPUT: integer, the number of vertices.

        EXAMPLES::

        This is the size parameter in the `Flag` initialization ::

            sage: from sage.algebras.flag_algebras import *
            sage: GraphTheory(4).size()
            4
        """
        return self._n
    
    vertex_number = size
    
    cpdef blocks(self, as_tuple=False, key=None, optional=0):
        r"""
        Returns the blocks

        INPUT:

        - ``as_tuple`` -- boolean (default: `False`); if the result should
            contain the blocks as a tuple

        OUTPUT: A dictionary, one entry for each element in the signature
            and list (or tuple) of the blocks for that signature.
        """
        if optional==0:
            reblocks = self._blocks
        elif optional==1:
            reblocks = self._blocks_optional
        else:
            reblocks = self._blocks | self._blocks_optional
        if as_tuple:
            if key != None:
                return tuple([tuple(yy) for yy in reblocks[key]])
            ret = {}
            for xx in reblocks:
                ret[xx] = tuple([tuple(yy) for yy in reblocks[xx]])
            return ret
        if key!=None:
            return reblocks[key]
        return reblocks

    cpdef ftype(self):
        r"""
        Returns the ftype of this `Flag`

        EXAMPLES::

        Ftype of a pointed triangle is just a point ::

            sage: from sage.algebras.flag_algebras import *
            sage: pointed_triangle = GraphTheory(3, edges=[[0, 1], [0, 2], [1, 2]], ftype=[0])
            sage: pointed_triangle.ftype()
            Ftype on 1 points with edges=[]
        
        And with two points it is ::
        
            sage: two_pointed_triangle = GraphTheory(3, edges=[[0, 1], [0, 2], [1, 2]], ftype=[0, 1])
            sage: two_pointed_triangle.ftype()
            Ftype on 2 points with edges=[[0, 1]]

        .. NOTE::

            This is essentially the subflag, but the order of points matter. The result is saved
            for speed.

        .. SEEALSO::

            :func:`subflag`
        """
        if self._ftype==None:
            if self.is_ftype():
                self._ftype = self
            self._ftype = self.subflag([])
        return self._ftype
    
    cpdef ftype_points(self):
        r"""
        The points of the ftype inside self.
        
        This gives an injection of ftype into self

        OUTPUT: list of integers

        EXAMPLES::

            sage: from sage.algebras.flag_algebras import *
            sage: two_pointed_triangle = GraphTheory(3, edges=[[0, 1], [0, 2], [1, 2]], ftype=[0, 1])
            sage: two_pointed_triangle.ftype_points()
            [0, 1]

        .. SEEALSO::

            :func:`__init__`
        """
        return self._ftype_points
    
    cpdef not_ftype_points(self):
        r"""
        This is a helper function, caches the points that are not
        part of the ftype.
        """
        if self._not_ftype_points != None:
            return self._not_ftype_points
        self._not_ftype_points = [ii for ii in range(self.size()) if ii not in self._ftype_points]
        return self._not_ftype_points
    
    def unique(self, weak=False):
        r"""
        This returns a unique identifier that can equate isomorphic
        objects

        EXAMPLES::

        Isomorphic graphs have the same :func:`unique` value ::

            sage: from sage.algebras.flag_algebras import *
            sage: b1 = [[0, 1], [0, 2], [0, 4], [1, 3], [2, 4]]
            sage: b2 = [[0, 4], [1, 2], [1, 3], [2, 3], [3, 4]]
            sage: g1 = GraphTheory(5, edges=b1)
            sage: g2 = GraphTheory(5, edges=b2)
            sage: g1.unique() == g2.unique()
            True
            
        .. NOTE::

            The value returned here depends on the values of
            the parent `CombinatorialTheory`

        .. SEEALSO::

            :func:`theory`
            :func:`CombinatorialTheory.identify`
            :func:`__eq__`
        """
        if not self._pure_flag:
            raise ValueError("Unique is not defined for pure flags")
        if weak:
            return self.theory().identify(self._n, [self._ftype_points], **self._blocks)
        if self._unique==():
            self._unique = self.theory().identify(
                self._n, self._ftype_points, **self._blocks)
        return self._unique
    
    cpdef is_ftype(self):
        r"""
        Returns `True` if this flag is an ftype.

        .. SEEALSO::

            :func:`_repr_`
        """
        return self._n == self._ftype_size

    cpdef is_flag(self):
        return self._pure_flag

    cpdef is_pattern(self):
        return not self._pure_flag

    def _add_(self, other):
        r"""
        Add two Flags together
        
        The flags must have the same ftype. Different sizes are 
            all shifted to the larger one.

        OUTPUT: The :class:`FlagAlgebraElement` object, 
            which is the sum of the two parameters

        EXAMPLES::

        Adding to self is 2*self ::

            sage: from sage.algebras.flag_algebras import *
            sage: g = GraphTheory(3)
            sage: g+g==2*g
            True
        
        Adding two distinct elements with the same size gives a vector 
        with exactly two `1` entries ::

            sage: h = GraphTheory(3, edges=[[0, 1]])
            sage: (g+h).values()
            (1, 1, 0, 0)
        
        Adding with different size the smaller flag
        is shifted to have the same size ::
        
            sage: e = GraphTheory(2)
            sage: (e+h).values()
            (1, 5/3, 1/3, 0)

        .. SEEALSO::

            :func:`FlagAlgebraElement._add_`
            :func:`__lshift__`

        """
        if self.ftype()!=other.ftype():
            raise TypeError("The terms must have the same ftype")
        return self.afae()._add_(other.afae())
    
    def _sub_(self, other):
        r"""
        Subtract a Flag from `self`
        
        The flags must have the same ftype. Different sizes are 
            all shifted to the larger one.

        EXAMPLES::
            
            sage: from sage.algebras.flag_algebras import *
            sage: g = GraphTheory(2)
            sage: h = GraphTheory(3, edges=[[0, 1]])
            sage: (g-h).values()
            (1, -1/3, 1/3, 0)

        .. SEEALSO::

            :func:`_add_`
            :func:`__lshift__`
            :func:`FlagAlgebraElement._sub_`

        """
        if self.ftype()!=other.ftype():
            raise TypeError("The terms must have the same ftype")
        return self.afae()._sub_(other.afae())
    
    def _mul_(self, other):
        r"""
        Multiply two flags together.
        
        The flags must have the same ftype. The result
        will have the same ftype and size 
        `self.size() + other.size() - self.ftype().size()`

        OUTPUT: The :class:`FlagAlgebraElement` object, 
            which is the product of the two parameters

        EXAMPLES::

        Pointed edge multiplied by itself ::

            sage: from sage.algebras.flag_algebras import *
            sage: pe = GraphTheory(2, edges=[[0, 1]], ftype=[0])
            sage: (pe*pe).values()
            (0, 0, 0, 0, 1, 1)

        .. SEEALSO::

            :func:`FlagAlgebraElement._mul_`
            :func:`mul_project`
            :func:`CombinatorialTheory.mul_project_table`

        TESTS::

            sage: sum((pe*pe*pe*pe).values())
            11
            sage: e = GraphTheory(2)
            sage: (e*e).values()
            (1, 2/3, 1/3, 0, 2/3, 1/3, 0, 0, 1/3, 0, 0)
        """
        if self.ftype()!=other.ftype():
            raise TypeError("The terms must have the same ftype")
        return self.afae()._mul_(other.afae())
    
    def __lshift__(self, amount):
        r"""
        `FlagAlgebraElement`, equal to this, with size is shifted by the amount

        EXAMPLES::

        Edge shifted to size `3` ::

            sage: from sage.algebras.flag_algebras import *
            sage: edge = GraphTheory(2, edges=[[0, 1]])
            sage: (edge<<1).values()
            (0, 1/3, 2/3, 1)

        .. SEEALSO::

            :func:`FlagAlgebraElement.__lshift__`
        """
        return self.afae().__lshift__(amount)
    
    def __truediv__(self, other):
        r"""
        Divide by a scalar

        INPUT:

        - ``other`` -- number; any number such that `1` can be divided with that

        OUTPUT: The `FlagAlgebraElement` resulting from the division

        EXAMPLES::

        Divide by `2` ::

            sage: from sage.algebras.flag_algebras import *
            sage: g = GraphTheory(3)
            sage: (g/2).values()
            (1/2, 0, 0, 0)
            
        Even for `x` symbolic `1/x` is defined, so the division is understood ::
            sage: var('x')
            x
            sage: g = GraphTheory(2)
            sage: g/x
            Flag Algebra Element over Symbolic Ring
            1/x - Flag on 2 points, ftype from [] with edges=[]
            0   - Flag on 2 points, ftype from [] with edges=[[0, 1]]
        
        .. NOTE::

            Dividing by `Flag` or `FlagAlgebraElement` is not allowed, only
            numbers such that the division is defined in some extension
            of the rationals.

        .. SEEALSO::

            :func:`FlagAlgebraElement.__truediv__`
        """
        return self.afae().__truediv__(other)
    
    def __eq__(self, other):
        r"""
        Compare two flags for == (equality)
        
        This is the isomorphism defined by the identifiers,
        respecting the types.

        .. SEEALSO::

            :func:`unique`
            :func:`theory`
            :func:`CombinatorialTheory.identify`
        """
        if type(other)!=type(self):
            return False
        if self.parent()!=other.parent():
            return False
        return self.unique() == other.unique()
    
    def weak_eq(self, other):
        r"""
        Compare two flags for weak equality
        
        This is the isomorphism where type permutations
        are allowed.

        .. SEEALSO::

            :func:`unique`
            :func:`theory`
            :func:`CombinatorialTheory.identify`
        """
        if type(other)!=type(self):
            return False
        if self.parent()!=other.parent():
            return False
        return self.unique(weak=True) == other.unique(weak=True)
    
    def __lt__(self, other):
        r"""
        Compare two flags for < (proper induced inclusion)
        
        Returns true if self appears as a proper induced structure 
        inside other.

        .. SEEALSO::

            :func:`__le__`
        """
        if type(other)!=type(self):
            return False
        if self.parent()!=other.parent():
            return False
        if self.size()>=other.size():
            return False
        if self.ftype() != other.ftype():
            return False
        for subp in itertools.combinations(other.not_ftype_points(), self.size()-self.ftype().size()):
            sig_check()
            osub = other.subflag(subp)
            if osub==None or osub.unique()==None:
                continue
            if osub.unique()==self.unique():
                return True
        return False
    
    def __le__(self, other):
        r"""
        Compare two flags for <= (induced inclusion)
        
        Returns true if self appears as an induced structure inside
        other.

        EXAMPLES::

        Edge appears in a 4 star ::

            sage: from sage.algebras.flag_algebras import *
            sage: star = GraphTheory(4, edges=[[0, 1], [0, 2], [0, 3]])
            sage: edge = GraphTheory(2, edges=[[0, 1]])
            sage: edge <= star
            True
            
        The ftypes must agree ::
        
            sage: p_edge = GraphTheory(2, edges=[[0, 1]], ftype_points=[0])
            sage: p_edge <= star
            False
        
        But when ftypes agree, the inclusion must respect it ::
            
            sage: pstar = star.subflag(ftype_points=[0])
            sage: sub1 = GraphTheory(3, ftype=[0], edges=[[0, 1], [0, 2]])
            sage: sub1 <= pstar
            True
            sage: sub2 = GraphTheory(3, ftype=[1], edges=[[0, 1], [0, 2]])
            sage: sub2 <= pstar
            False

        .. SEEALSO::

            :func:`__lt__`
            :func:`__eq__`
            :func:`unique`
        """
        return self==other or self<other
    
    def __hash__(self):
        r"""
        A hash based on the unique identifier
        so this is compatible with `__eq__`.
        """
        return hash(self.unique())
    
    def __getstate__(self):
        r"""
        Saves this flag to a dictionary
        """
        dd = {'theory': self.theory(),
              'n': self._n, 
              'ftype_points': self._ftype_points, 
              'blocks':self._blocks, 
              'blocks_optional': self._blocks_optional,
              'unique':self._unique}
        return dd
    
    def __setstate__(self, dd):
        r"""
        Loads this flag from a dictionary
        """
        self._set_parent(dd['theory'])
        self._n = dd['n']
        self._ftype_points = dd['ftype_points']
        self._ftype_size = len(self._ftype_points)
        self._not_ftype_points = None
        self._blocks = dd['blocks']
        self._unique = dd['unique']
        if 'blocks_optional' in dd.keys():
            self._blocks_optional = dd['blocks_optional']
        if 'pure' in dd.keys():
            self._pure_flag = dd['pure']
    
    def project(self, ftype_inj=tuple()):
        r"""
        Project this `Flag` to a smaller ftype
        
        
        INPUT:

        - ``ftype_inj`` -- tuple (default: (, )); the injection of the
            projected ftype inside the larger ftype

        OUTPUT: the `FlagAlgebraElement` resulting from the projection

        EXAMPLES::

        If the center of a cherry is flagged, then the projection has
        coefficient 1/3 ::

            sage: from sage.algebras.flag_algebras import *
            sage: p_cherry = GraphTheory(3, edges=[[0, 1], [0, 2]], ftype_points=[0])
            sage: p_cherry.project().values()
            (0, 0, 1/3, 0)

        .. NOTE::

            If `ftype_inj==tuple(range(self.ftype().size()))` then this
            does nothing.

        .. SEEALSO::

            :func:`FlagAlgebraElement.project`
        """
        return self.afae().project(ftype_inj)
    
    def mul_project(self, other, ftype_inj=tuple()):
        r"""
        Multiply self with other, and the project the result.

        INPUT:

        - ``ftype_inj`` -- tuple (default: (, )); the injection of the
            projected ftype inside the larger ftype

        OUTPUT: the `FlagAlgebraElement` resulting from the multiplication
            and projection

        EXAMPLES::

        Pointed edge multiplied with itself and projected ::

            sage: from sage.algebras.flag_algebras import *
            sage: p_edge = GraphTheory(2, edges=[[0, 1]], ftype_points=[0])
            sage: p_edge.mul_project(p_edge).values()
            (0, 0, 1/3, 1)

        .. NOTE::

            If `ftype_inj==tuple(range(self.ftype().size()))` then this
            is the same as usual multiplication.

        .. SEEALSO::

            :func:`_mul_`
            :func:`project`
            :func:`FlagAlgebraElement.mul_project`
        """
        return self.afae().mul_project(other, ftype_inj)
    
    def density(self, other):
        r"""
        The density of self in other.
        
        Randomly choosing self.size() points in other, the
        probability of getting self.

        EXAMPLES::

        Density of an edge in the cherry graph is 2/3 ::

            sage: from sage.algebras.flag_algebras import *
            sage: cherry = GraphTheory(3, edges=[[0, 1], [0, 2]])
            sage: edge = GraphTheory(2, edges=[[0, 1]])
            sage: cherry.density(edge)
            2/3
        
        .. SEEALSO::
        
            :func:`FlagAlgebraElement.density`
        """
        safae = self.afae()
        oafae = safae.parent(other)
        return self.afae().density(other)
    
    def _ftypes_inside(self, target):
        r"""
        Returns the possible ways self ftype appears in target

        INPUT:

        - ``target`` -- Flag; the flag where we are looking for copies of self

        OUTPUT: list of Flags with ftype matching as self, not necessarily unique
        """
        if not self._pure_flag:
            raise ValueError("Can't list ftypes in a patter")
        ret = []
        lrp = list(range(target.size()))
        for ftype_points in itertools.permutations(range(target.size()), self._n):
            sig_check()
            if target.subflag(ftype_points, ftype_points)==self:
                ret.append(target.subflag(lrp, ftype_points))
        return ret
    
    cpdef densities(self, n1, n1flgs, n2, n2flgs, ftype_remap, large_ftype, small_ftype):
        r"""
        Returns the density matrix, indexed by the entries of `n1flgs` and `n2flgs`
        
        The matrix returned has entry `(i, j)` corresponding to the possibilities of
        `n1flgs[i]` and `n2flgs[j]` inside self, projected to the small ftype. 
        
        This is the same as counting the ways we can choose `n1` and `n2` points
        inside `self`, such that the two sets cover the entire `self.size()` point
        set, and calculating the probability that the overlap induces an ftype
        isomorphic to `large_ftype` and the points sets are isomorphic to 
        `n1flgs[i]` and `n2flgs[j]`.

        INPUT:

        - ``n1`` -- integer; the size of the first flag list
        - ``n1flgs`` -- list of flags; the first flag list (each of size `n1`)
        - ``n2`` -- integer; the size of the second flag list
        - ``n2flgs`` -- list of flags; the second flag list (each of size `n2`)
        - ``ftype_remap`` -- list; shows how to remap `small_ftype` into `large_ftype`
        - ``large_ftype`` -- ftype; the ftype of the overlap
        - ``small_ftype`` -- ftype; the ftype of self

        OUTPUT: a sparse matrix corresponding with the counts

        .. SEEALSO::

            :func:`CombinatorialTheory.mul_project_table`
            :func:`FlagAlgebra.mul_project_table`
            :func:`FlagAlgebraElement.mul_project`
        """
        cdef int N = self._n
        cdef int small_size = small_ftype.size()
        cdef int large_size = large_ftype.size()
        cdef int ctr = 0
        cdef bint chk = 0
        cdef int valid_ftypes = 0
        cdef int correct_ftypes = 0
        cdef int valid_flag_pairs = 0
        
        ret = {}
        small_points = self._ftype_points
        for difference in itertools.permutations(self.not_ftype_points(), large_size - small_size):
            sig_check()
            large_points = [0]*len(ftype_remap)
            for ii in range(len(ftype_remap)):
                vii = ftype_remap[ii]
                if vii<small_size:
                    large_points[ii] = small_points[vii]
                else:
                    large_points[ii] = difference[vii-small_size]
            ind_large_ftype = self.subflag([], ftype_points=large_points)
            if ind_large_ftype.unique()==None:
                continue
            
            valid_ftypes += 1
            if ind_large_ftype==large_ftype:
                correct_ftypes += 1
                not_large_points = [ii for ii in range(N) if ii not in large_points]
                for n1_extra_points in itertools.combinations(not_large_points, n1 - large_size):
                    n1_subf = self.subflag(n1_extra_points, ftype_points=large_points)
                    if n1_subf.unique()==None:
                        continue
                    try:
                        n1_ind = n1flgs.index(n1_subf)
                    except ValueError:
                        raise ValueError("Could not find \n", n1_subf, "\nin the list of ", \
                                         n1, " sized flags with ", large_ftype, \
                                         ".\nThis can happen if the generator and identifier ",\
                                         "(from the current CombinatorialTheory) is incompatible, ",\
                                         "or if the theory is not heredetary")
                    
                    remaining_points = [ii for ii in not_large_points if ii not in n1_extra_points]
                    for n2_extra_points in itertools.combinations(remaining_points, n2 - large_size):
                        n2_subf = self.subflag(n2_extra_points, ftype_points=large_points)
                        if n2_subf.unique()==None:
                            continue
                        valid_flag_pairs += 1
                        try:
                            n2_ind = n2flgs.index(n2_subf)
                        except:
                            raise ValueError("Could not find \n", n2_subf, "\nin the list of ", \
                                             n2, " sized flags with ", large_ftype, \
                                             ".\nThis can happen if the generator and identifier ",\
                                             "(from the current CombinatorialTheory) is incompatible, ",\
                                             "or if the theory is not heredetary")
                        try:
                            ret[(n1_ind, n2_ind)] += 1
                        except:
                            ret[(n1_ind, n2_ind)] = 1
        return (len(n1flgs), len(n2flgs), ret, valid_ftypes, valid_flag_pairs, correct_ftypes)

In [ ]:
class FlagPattern:
    cdef int _n
    cdef int _ftype_size
    
    cdef list _ftype_points
    cdef list _not_ftype_points
    cdef dict _blocks
    cdef dict _blocks_optional
    cdef tuple _unique
    cdef CombinatorialTheory _theory
    
    cdef Flag _ftype
    
    def __init__(self, theory, n, **params):
        self._n = int(n)
        
        if 'ftype_points' in params:
            ftype_points = params['ftype_points']
        elif 'ftype' in params:
            ftype_points = params['ftype']
        else:
            ftype_points = []
        
        self._ftype_size = len(ftype_points)
        self._ftype_points = list(ftype_points)
        self._not_ftype_points = None
        self._blocks = {}
        self._blocks_optional = {}
        for xx in theory._signature.keys():
            self._blocks[xx] = []
            if xx in params:
                self._blocks[xx] = [list(yy) for yy in params[xx]]
            self._blocks_optional[xx] = []
            for xx_opti in [xx+"_o", xx+"_optional", xx+"_opti", "o_"+xx, "optional_"+xx, "opti_"+xx]:
                if xx_opti in params:
                    xx_oblocks = [list(yy) for yy in params[xx_opti]]
                    for ed in xx_oblocks:
                        if len(_subblock_helper(self._ftype_points, xx_oblocks))!=0:
                            raise ValueError("Can't have optional blocks in ftype")
                    self._blocks_optional[xx] = xx_oblocks
                    break
        self._unique = ()
        self._ftype = None
        self._theory = theory
    
    def _repr_(self):
        #TODO
        blocks = self.blocks()
        strblocks = ', '.join([xx+'='+str(blocks[xx]) for xx in blocks.keys()])
        if self.is_ftype():
            return 'Ftype on {} points with {}'.format(self.size(), strblocks)
        return 'Flag on {} points, ftype from {} with {}'.format(self.size(), self.ftype_points(), strblocks)
    
    def compact_repr(self):
        #TODO
        blocks = self.blocks()
        ret = ["n:{}".format(self.size())]
        if len(self._ftype_points)!=0:
            ret.append("t:"+"".join(map(str, self._ftype_points)))
        for name in self.theory()._signature.keys():
            desc = name + ":"
            arity = self.theory()._signature[name]
            if arity==1:
                desc += "".join([str(xx[0]) for xx in blocks[name]])
            else:
                desc += ",".join(["".join(map(str, ed)) for ed in blocks[name]])
            ret.append(desc)
        return "; ".join(ret)
            
    
    def raw_numbers(self):
        #TODO
        numbers = [self.size()] + self.ftype_points() + [15]
        blocks = self.blocks()
        for xx in blocks:
            for yy in blocks[xx]:
                numbers += yy
            numbers.append(15)
        return numbers
    
    def combinatorial_theory(self):
        return self._theory
    
    theory = combinatorial_theory
    
    def is_compatible_flag(self, flag):
        if self._n > flag._n:
            return False
        if self._theory != flag._theory:
            return False
        if self.ftype() != flag.
        for perm in itertools.permutations(range(flag._n), self._n):
            
    
    def compatible_flags(self):
        ret = []
        for xx in 
    
    def as_flag_algebra_element(self, basis=QQ):
        from sage.algebras.flag_algebras import FlagAlgebra
        targ_alg = FlagAlgebra(basis, self.theory(), self.ftype())
        return targ_alg(self)
    
    afae = as_flag_algebra_element
    
    def as_operand(self):
        return self.afae(QQ)
    
    cpdef size(self):
        return self._n
    
    vertex_number = size
    
    cpdef blocks(self, as_tuple=False, key=None):
        if as_tuple:
            if key != None:
                return tuple([tuple(yy) for yy in self._blocks[key]])
            ret = {}
            for xx in self._blocks:
                ret[xx] = tuple([tuple(yy) for yy in self._blocks[xx]])
            return ret
        if key!=None:
            return self._blocks[key]
        return self._blocks
    
    cpdef ftype(self):
        if self._ftype==None:
            if self.is_ftype():
                self._ftype = self
            self._ftype = self.subflag([])
        return self._ftype
    
    cpdef ftype_points(self):
        return self._ftype_points
    
    cpdef not_ftype_points(self):
        if self._not_ftype_points != None:
            return self._not_ftype_points
        self._not_ftype_points = [ii for ii in range(self.size()) if ii not in self._ftype_points]
        return self._not_ftype_points
    
    def unique(self, weak=False):
        if weak:
            return self.theory().identify(self._n, [self._ftype_points], **self._blocks)
        if self._unique==():
            self._unique = self.theory().identify(
                self._n, self._ftype_points, **self._blocks)
        return self._unique
    
    cpdef is_ftype(self):
        return self._n == self._ftype_size
    
    def _add_(self, other):
        if self.ftype()!=other.ftype():
            raise TypeError("The terms must have the same ftype")
        return self.afae()._add_(other.afae())
    
    def _sub_(self, other):
        if self.ftype()!=other.ftype():
            raise TypeError("The terms must have the same ftype")
        return self.afae()._sub_(other.afae())
    
    def _mul_(self, other):
        if self.ftype()!=other.ftype():
            raise TypeError("The terms must have the same ftype")
        return self.afae()._mul_(other.afae())
    
    def __lshift__(self, amount):
        return self.afae().__lshift__(amount)
    
    def __truediv__(self, other):
        return self.afae().__truediv__(other)
    
    def __eq__(self, other):
        if type(other)!=type(self):
            return False
        if self.parent()!=other.parent():
            return False
        return self.unique() == other.unique()
    
    def weak_eq(self, other):
        if type(other)!=type(self):
            return False
        if self.parent()!=other.parent():
            return False
        return self.unique(weak=True) == other.unique(weak=True)
    
    def __lt__(self, other):
        if type(other)!=type(self):
            return False
        if self.parent()!=other.parent():
            return False
        if self.size()>=other.size():
            return False
        if self.ftype() != other.ftype():
            return False
        for subp in itertools.combinations(other.not_ftype_points(), self.size()-self.ftype().size()):
            sig_check()
            osub = other.subflag(subp)
            if osub==None or osub.unique()==None:
                continue
            if osub.unique()==self.unique():
                return True
        return False
    
    def __le__(self, other):
        return self==other or self<other
    
    def __hash__(self):
        return hash(self.unique())
    
    def __getstate__(self):
        dd = {'theory': self.theory(),
              'n': self._n, 
              'ftype_points': self._ftype_points, 
              'blocks':self._blocks, 
              'unique':self._unique}
        return dd
    
    def __setstate__(self, dd):
        self._set_parent(dd['theory'])
        self._n = dd['n']
        self._ftype_points = dd['ftype_points']
        self._ftype_size = len(self._ftype_points)
        self._not_ftype_points = None
        self._blocks = dd['blocks']
        self._unique = dd['unique']
    
    def project(self, ftype_inj=tuple()):
        return self.afae().project(ftype_inj)
    
    def mul_project(self, other, ftype_inj=tuple()):
        return self.afae().mul_project(other, ftype_inj)
    
    def density(self, other):
        safae = self.afae()
        oafae = safae.parent(other)
        return self.afae().density(other)
    
    def _ftypes_inside(self, target):
        ret = []
        lrp = list(range(target.size()))
        for ftype_points in itertools.permutations(range(target.size()), self._n):
            sig_check()
            if target.subflag(ftype_points, ftype_points)==self:
                ret.append(target.subflag(lrp, ftype_points))
        return ret
    

In [72]:
sys.setprofile(None)

def tracefunc(frame, event, arg, indent=[0]):
    if event == "call":
        indent[0] += 2
        module_name = inspect.getmodule(frame.f_code).__name__ if inspect.getmodule(frame.f_code) else "unknown"
        class_name = ""
        try:
            class_name = inspect.getclasstree(inspect.getmro(type(frame.f_locals['self'])), True)[-1][0][0].__name__
        except:
            if 'self' in frame.f_locals:
                class_name = str(inspect.getclasstree(inspect.getmro(type(frame.f_locals['self'])), True))
            else:
                class_name = ""
        function_name = frame.f_code.co_name
        tst = module_name + function_name
        if "IPython" in tst or "enum" in tst or "typing" in tst or "traitlets" in tst or "util" in tst or "ipykernel" in tst:
            pass
        else:
            if "unknown" in tst or "tokenize" in tst or "logging" in tst or "json" in tst or "preparse" in tst or indent[0]<=0:
                pass
            else:
                print("-" * indent[0] + "> call function", function_name, "in", class_name, "from", module_name)
    elif event == "return":
        indent[0] -= 2
    return tracefunc

#sys.setprofile(tracefunc)

#asd = sum([G(0)])

In [3]:
#for quick sparse matrix printing
def print_sparse(ls, eps=1e-6):
    nzs = [(ii, ls[ii]) for ii in range(len(ls)) if abs(ls[ii])>eps]
    if isinstance(ls[0], Rational):
        st = "\n".join(["{}: {}".format(nn[0], nn[1]) for nn in nzs])
    else:
        st = "\n".join(["{}: {:.4f}".format(nn[0], float(nn[1])) for nn in nzs])
    return st

#for debug printing values with their variable names
def debug(*args):
    import inspect
    frame = inspect.currentframe().f_back
    s = inspect.getframeinfo(frame).code_context[0]
    r = s.split('(')[1].split(')')[0].split(',')
    names = [name.strip() for name in r]
    
    for name, value in zip(names, args):
        if isinstance(value, str):
            print(value)
        else:
            print(f"{name}: {value}")


In [14]:
def verify_solution(self, solution, target_element, target_size, maximize=True, positives=None, construction=None):
    #
    # Checking eigenvalues and positivity constraints
    #
    
    if len(solution)==2 and solution[0] in QQ:
        solution = solution[1]
    if len(solution)==3 and solution[0] in QQ:
        solution = solution[1]
    
    if len(solution[-1])>0 and min(solution[-1])<0:
        print("Solution is not valid!")
        print("Linear constraint's coefficient is negative {}".format(min(solution[-1])))
        return -1

    for ii,X in enumerate(solution[:-1]):
        if min(X.eigenvalues())<0:
            print("Solution is not valid!")
            print("Matrix {} is not semidefinite: {}".format(ii, min(X.eigenvalues())))
            return -1
    
    print("Solution matrices are all semidefinite, linear coefficients are all non-negative")

    #
    # Initial setup
    #
    
    mult = -1 if maximize else 1
    base_flags = self.generate_flags(target_size)
    target_vector_exact = (target_element.project()*(mult)<<(target_size - target_element.size())).values()
    if target_element.ftype().size()==0:
        one_vector = vector([1]*len(base_flags))
    else:
        one_vector = (target_element.ftype().project()<<(target_size - target_element.ftype().size())).values()
    
    #
    # Create the types used in the calculation
    #
    
    ftype_data = []
    for fs in range(target_size-2, 1, -2):
        for fl in self.generate_flags(fs):
            ftype = fl.subflag(ftype_points=list(range(fs)))
            ns = (target_size + fs)/2
            ftype_data.append((ns, ftype, target_size))
    ftype_data.sort()
    
    print("Done calculating types relevant for the calculation")
    
    #
    # Create the semidefinite matrix data
    #
    
    table_list = []
    for ii, dat in enumerate(ftype_data):
        ns, ftype, target_size = dat
        #calculate the table
        table = self.mul_project_table(ns, ns, ftype, ftype_inj=[], target_size=target_size)
        if table!=None:
            table_list.append(table)
        print("Done with mult table for {}".format(ftype))

    print("Done calculating semidefinite constraints")
    
    #
    # Create the data from linear constraints
    #

    positives_list_exact = []
    if positives != None:
        for ii, fv in enumerate(positives):
            if isinstance(fv, Flag):
                continue
            nf = fv.size()
            df = target_size + fv.ftype().size() - nf
            mult_table = self.mul_project_table(nf, df, fv.ftype(), ftype_inj=[], target_size=target_size)
            fvvals = fv.values()
            m = matrix(QQ, [vector(fvvals*mat) for mat in mult_table])
            positives_list_exact += list(m.T)
            print("Done with positivity constraint {}".format(ii))
    positives_matrix_exact = matrix(QQ, len(positives_list_exact), len(base_flags), positives_list_exact)
    
    print("Done calculating linear constraints")

    #
    # Calculate the bound the solution provides
    #
    
    slacks = target_vector_exact - positives_matrix_exact.T*solution[-1]
    for ii, table in enumerate(table_list):
        for gg, mat_gg in enumerate(table):
            slacks[gg] -= sum([mat_gg.rows()[jj]*solution[ii][jj] for jj in range(mat_gg.nrows())])
    res = min(slacks)*mult
    
    print("The solution is valid, it proves the bound {}".format(res))
    
    return res

In [15]:
verify_solution(G, res, G(2), 4)

Solution matrices are all semidefinite, linear coefficients are all non-negative
Done calculating types relevant for the calculation
Done with mult table for Ftype on 2 points with edges=[]
Done with mult table for Ftype on 2 points with edges=[[0, 1]]
Done calculating semidefinite constraints
Done calculating linear constraints
The solution is valid, it proves the bound 2/3


2/3

In [2]:
G = GraphTheory
G.exclude(G(3))
G.optimize(G(3, edges=[[0, 1], [0, 2]], ftype=[0, 1]), 4, exact=True, construction=[])

Base flags generated, their number is 7
The relevant ftypes are constructed, their number is 2
Block sizes before symmetric/asymmetric change is applied: [3, 4]


Done with mult table for Ftype on 2 points with edges=[[0, 1]]: : 2it [00:00, 687.70it/s]

Tables finished
Constraints finished
Running sdp without construction. Used block sizes are [2, 1, 3, 1, -7, -2]
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 1.00e+00 Pobj: -2.4031147e+01 Ad: 6.90e-01 Dobj:  2.6085342e-02 
Iter:  2 Ap: 1.00e+00 Pobj: -2.3311969e+01 Ad: 9.45e-01 Dobj: -1.0244791e-01 
Iter:  3 Ap: 1.00e+00 Pobj: -1.0098255e+01 Ad: 8.61e-01 Dobj: -9.5774050e-02 
Iter:  4 Ap: 1.00e+00 Pobj: -2.8128533e+00 Ad: 7.87e-01 Dobj: -9.7496556e-02 
Iter:  5 Ap: 1.00e+00 Pobj: -5.2363527e-01 Ad: 8.86e-01 Dobj: -1.0118969e-01 
Iter:  6 Ap: 1.00e+00 Pobj: -2.2795982e-01 Ad: 9.03e-01 Dobj: -1.3241207e-01 
Iter:  7 Ap: 1.00e+00 Pobj: -1.9299418e-01 Ad: 8.27e-01 Dobj: -1.6442902e-01 
Iter:  8 Ap: 1.00e+00 Pobj: -1.7392267e-01 Ad: 8.80e-01 Dobj: -1.7004887e-01 
Iter:  9 Ap: 1.00e+00 Pobj: -1.7178533e-01 Ad: 1.00e+00 Dobj: -1.7148966e-01 
Iter: 10 Ap: 9.96e-01 Pobj: -1.7158265e-01 Ad: 1.00e+00 Dobj: -1.7157022e-01 
Iter: 11 Ap: 9.93e

Rounding X matrices


100%|████████████████████████████████████████████| 4/4 [00:00<00:00, 150.82it/s]


Calculating resulting bound


100%|████████████████████████████████████████████| 2/2 [00:00<00:00, 132.66it/s]

Rounding errors are [0.00046531961922422746, 2.656989831040235e-06, 0.0005193140193275126, 1.2871523029645004e-06, 0]


11/64

In [3]:
(11/64).n()

0.171875000000000

In [38]:
def _round_sdp_solution_no_phi(self, sdp_result, sdp_data, table_constructor, \
                               constraints_data, denom=1024):
    from tqdm import tqdm
    import numpy as np
    from numpy import linalg as LA
    from sage.functions.other import ceil
    from sage.matrix.special import diagonal_matrix

    #unpack variables

    block_sizes, target_list_exact, mat_inds, mat_vals = sdp_data
    target_vector_exact = vector(target_list_exact)
    flags_num, constraints_vals, positives_list_exact, one_vector = constraints_data
    positives_matrix_exact = matrix(QQ, len(positives_list_exact), flags_num, positives_list_exact)

    one_vector_exact = positives_matrix_exact.rows()[-2]
    positives_matrix_exact = positives_matrix_exact[:-2, :] # remove the equality constraints

    flags_num = -block_sizes[-2] # same as |F_n|

    X_matrices_approx = sdp_result['X'][:-2]
    X_matrices_rounded = []
    print("Rounding X matrices")
    for X in tqdm(X_matrices_approx):

        Xr = _round_matrix(X, method=0, denom=denom)
        Xnp = np.array(Xr)
        eigenvalues, eigenvectors = LA.eig(Xnp)
        emin = min(eigenvalues)
        if emin<0:
            eminr = ceil(-emin*denom)/denom
            Xr = matrix(QQ, Xr) + diagonal_matrix(QQ, [eminr]*len(X), sparse=True)
        X_matrices_rounded.append(Xr)
    X_matrices_flat = [vector(_flatten_matrix(X.rows(), doubled=False)) for X in (X_matrices_rounded)]

    e_vector_approx = sdp_result['X'][-1][:-2]
    e_vector_rounded = vector(_round_list(e_vector_approx, force_pos=True, method=0, denom=denom))
    
    phi_vector_approx = sdp_result['y']
    phi_vector_rounded = vector(_round_list(phi_vector_approx, force_pos=True, method=0, denom=denom))

    slacks = target_vector_exact - positives_matrix_exact.T*e_vector_rounded
    block_index = 0
    print("Calculating resulting bound")
    for params in tqdm(table_constructor.keys()):
        ns, ftype, target_size = params
        table = self.mul_project_table(ns, ns, ftype, ftype_inj=[], target_size=target_size)
        for gg, morig in enumerate(table):
            for plus_index, base in enumerate(table_constructor[params]):
                block_dim = block_sizes[block_index + plus_index]
                X_flat = X_matrices_flat[block_index + plus_index]
                M = base * morig * base.T
                M_flat_vector_exact = vector(_flatten_matrix(M.rows(), doubled=True))
                slacks[gg] -= M_flat_vector_exact*X_flat
        block_index += len(table_constructor[params])
    result = min([slacks[ii]/oveii for ii, oveii in enumerate(one_vector_exact) if oveii!=0])
    slacks -= result*one_vector_exact
    
    rerrs = []
    for ii in range(len(X_matrices_rounded)):
        Xr = X_matrices_rounded[ii]
        Xo = matrix(sdp_result['X'][ii])
        rerrs.append((Xr-Xo).norm())
    rerrs.append((vector(e_vector_approx) - e_vector_rounded).norm())
    print("Rounding errors are {}".format(rerrs))
    
    return result, X_matrices_rounded, e_vector_rounded, slacks, [phi_vector_rounded]

def _round_sdp_solution_phi(self, sdp_result, sdp_data, table_constructor, \
                        constraints_data, phi_vectors_exact, denom=1024):
    r"""
    Round the SDP results output to get something exact.
    """
    
    #unpack variables
    block_sizes, target_list_exact, mat_inds, mat_vals = sdp_data
    target_vector_exact = vector(target_list_exact)
    flags_num, constraints_vals, positives_list_exact, one_vector = constraints_data
    positives_matrix_exact = matrix(QQ, len(positives_list_exact), flags_num, positives_list_exact)
    
    if len(phi_vectors_exact)==0:
        return None
    phi_vector_exact = phi_vectors_exact[0]

    one_vector_exact = positives_matrix_exact.rows()[-2]
    positives_matrix_exact = positives_matrix_exact[:-2, :] # remove the equality constraints
    
    flags_num = -block_sizes[-2] # same as |F_n|
    
    c_vector_approx = vector(sdp_result['X'][-2]) # dim: |F_n|, c vector, primal slack for flags
    c_vector_rounded = vector(_round_list(c_vector_approx, method=0, denom=denom)) # as above but rounded

    # The F (FF) flag indecies where the c vector is zero/nonzero
    c_zero_inds = [FF for FF, xx in enumerate(c_vector_approx) if (abs(xx)<1e-6 or phi_vector_exact[FF]!=0)]
    c_nonzero_inds = [FF for FF in range(flags_num) if FF not in c_zero_inds]



    positives_num = -block_sizes[-1] - 2 # same as m, number of positive constraints (-2 for the equality)

    phi_pos_vector_exact = positives_matrix_exact*phi_vector_exact # dim: m, witness that phi is positive

    e_vector_approx = vector(sdp_result['X'][-1][:-2]) # dim: m, the e vector, primal slack for positivitives
    e_vector_rounded = vector(_round_list(e_vector_approx, method=0, denom=denom)) # as above but rounded

    # The f (ff) positivity constraints where the e vector is zero/nonzero
    e_zero_inds = [ff for ff, xx in enumerate(e_vector_approx) if (abs(xx)<1e-6 or phi_pos_vector_exact[ff]!=0)]
    e_nonzero_inds = [ff for ff in range(positives_num) if ff not in e_zero_inds]



    bound_exact = target_vector_exact*phi_vector_exact 
    # the constraints for the flags that are exact
    corrected_target_relevant_exact = vector([target_vector_exact[FF] - bound_exact for FF in c_zero_inds])
    # the d^f_F matrix, but only the relevant parts for the rounding
    # so F where c_F = 0 and f where e_f != 0
    positives_matrix_relevant_exact = matrix(QQ, len(e_nonzero_inds), len(c_zero_inds), \
                                             [[positives_matrix_exact[ff][FF] for FF in c_zero_inds] for ff in e_nonzero_inds])
    # the e vector, but only the nonzero entries
    e_nonzero_list_rounded = [e_vector_rounded[ff] for ff in e_nonzero_inds]
    
    
    
    # 
    # Flatten the matrices relevant for the rounding
    # 
    # M table transforms to a matrix, (with nondiagonal entries doubled)
    # only the FF index matrices corresponding with tight constraints are used
    # 
    # X transforms to a vector
    # only the semidefinite blocks are used
    # 

    # The relevant entries of M flattened to a matrix this will be indexed by 
    # c_zero_inds and the triples from the types
    M_flat_relevant_matrix_exact = matrix(QQ, len(c_zero_inds), 0, 0, sparse=True)
    X_flat_vector_rounded = [] # The rounded X values flattened to a list
    block_index = 0
    block_info = []
    for params in table_constructor.keys():
        ns, ftype, target_size = params
        table = self.mul_project_table(ns, ns, ftype, ftype_inj=[], target_size=target_size)

        for plus_index, base in enumerate(table_constructor[params]):
            block_info.append([ftype, base])
            X_approx = sdp_result['X'][block_index + plus_index]
            X_flat_vector_rounded += _round_list(_flatten_matrix(X_approx), method=0, denom=denom)

            M_extra = []

            for FF in c_zero_inds:
                M_FF = table[FF]
                M_extra.append(_flatten_matrix((base * M_FF * base.T).rows(), doubled=True))

            M_flat_relevant_matrix_exact = M_flat_relevant_matrix_exact.augment(matrix(M_extra))
        block_index += len(table_constructor[params])


    # 
    # Append the relevant M matrix and the X with the additional values from
    # the positivity constraints. 
    #
    # Then correct the x vector values
    # 

    M_matrix_final = M_flat_relevant_matrix_exact.augment(positives_matrix_relevant_exact.T)
    x_vector_final = vector(X_flat_vector_rounded+e_nonzero_list_rounded)


    # Correct the values of the x vector, based on the minimal L_2 norm
    x_vector_corr = x_vector_final - M_matrix_final.T * \
    (M_matrix_final * M_matrix_final.T).pseudoinverse() * \
    (M_matrix_final*x_vector_final - corrected_target_relevant_exact) 
    
    #
    # Recover the X matrices and e vector from the corrected x
    #

    e_nonzero_vector_corr = x_vector_corr[-len(e_nonzero_inds):]
    e_vector_corr = vector(QQ, positives_num, dict(zip(e_nonzero_inds, e_nonzero_vector_corr)))
    if len(e_vector_corr)>0 and min(e_vector_corr)<0:
        print("Linear coefficient is negative: {}".format(min(e_vector_corr)))
        return None
    
    X_final = []
    slacks = target_vector_exact - positives_matrix_exact.T*e_vector_corr
    block_index = 0
    for params in table_constructor.keys():
        ns, ftype, target_size = params
        table = self.mul_project_table(ns, ns, ftype, ftype_inj=[], target_size=target_size)
        for plus_index, base in enumerate(table_constructor[params]):
            block_dim = block_sizes[block_index + plus_index]
            X_ii_small, x_vector_corr = _unflatten_matrix(x_vector_corr, block_dim)
            X_ii_small = matrix(X_ii_small)
            
            # verify semidefiniteness
            if not X_ii_small.is_positive_semidefinite():
                print("Rounded X matrix {} is not semidefinite: {}".format(block_index+plus_index, min(X_ii_small.eigenvalues())))
                return None
            
            # update slacks
            for gg, morig in enumerate(table):
                M = base * morig * base.T
                M_flat_vector_exact = vector(_flatten_matrix(M.rows(), doubled=True))
                slacks[gg] -= M_flat_vector_exact*vector(_flatten_matrix(X_ii_small.rows(), doubled=False))
            
            X_final.append(X_ii_small)
        block_index += len(table_constructor[params])
    
    result = min(slacks)
    slacks -= vector([result]*len(slacks))
    
    return result, X_final, e_vector_corr, slacks, phi_vectors_exact

def _format_optimizer_output(self, table_constructor, mult=1, sdp_output=None, rounding_output=None, file="default"):
    r"""
    Formats the outputs to a nice certificate
    
    The result contains: the final bound, the X matrices, the linear coefficients, the slacks,
                        a guess or exact construction, the list of base flags, the list of used (typed) flags
    """
    
    import json
    
    target_size = 0
    typed_flags = {}
    for params in table_constructor.keys():
        ns, ftype, target_size = params
        safekey = "target:" + str(ns) + "; " + self.flag_compact_repr(ftype)
        typed_flags[safekey] = [self.flag_compact_repr(xx) for xx in self.generate_flags(ns, ftype)]
    
    base_flags = [self.flag_compact_repr(xx) for xx in self.generate_flags(target_size)]
    
    result = None
    X_original = None
    e_vector = None
    slacks = None
    phi_vecs = None
    
    if sdp_output!=None:
        result = sdp_output['primal']
        X_original = [matrix(dat) for dat in sdp_output['X'][:-2]]
        e_vector = vector(sdp_output['X'][-1])
        slacks = vector(sdp_output['X'][-2])
        phi_vecs = [vector(sdp_output['y'])]
    elif rounding_output!=None:
        result, X_original, e_vector, slacks, phi_vecs = rounding_output
    
    result *= mult
    Ds, Ls = self._fix_X_bases(table_constructor, X_original)
    
    cert_dict = {"result": result, 
                 "D vectors": Ds,
                 "L matrices": Ls,
                 "e": e_vector,
                 "slacks": slacks,
                 "phi": phi_vecs,
                 "base flags": base_flags,
                 "typed flags": typed_flags
                }
    return cert_dict

def optimize(self, target_element, target_size, maximize=True, positives=None, \
                     construction=None, file=None, exact=False, denom=1024):
    from csdpy import solve_sdp
    from tqdm import tqdm
    import sys
    import io
    import time

    #
    # Initial setup
    #

    if target_size not in self.sizes():
        raise ValueError("For theory {}, size {} is not allowed.".format(self._name, target_size))

    base_flags = self.generate_flags(target_size)
    print("Base flags generated, their number is {}".format(len(base_flags)))
    mult = -1 if maximize else 1
    target_vector_exact = (target_element.project()*(mult)<<(target_size - target_element.size())).values()
    sdp_data = self._target_to_sdp_data(target_vector_exact)
    
    #
    # Create the relevant ftypes
    #
    
    ftype_data = self._get_relevant_ftypes(target_size)
    print("The relevant ftypes are constructed, their number is {}".format(len(ftype_data)))
    flags = [self.generate_flags(dat[0], dat[1]) for dat in ftype_data]
    flag_sizes = [len(xx) for xx in flags]
    print("Block sizes before symmetric/asymmetric change is applied: {}".format(flag_sizes))
    
    #
    # Create the table constructor and add it to sdp_data
    #
    
    table_constructor = self._create_table_constructor(ftype_data, target_size)
    sdp_data = self._tables_to_sdp_data(table_constructor, prev_data=sdp_data)
    print("Tables finished")

    #
    # Add constraints data and add it to sdp_data
    #
    
    constraints_data = self._create_constraints_data(positives, target_element, target_size)
    sdp_data = self._constraints_to_sdp_data(constraints_data, prev_data=sdp_data)
    print("Constraints finished")
    
    #
    # If construction is None or [] then run the optimizer without any construction
    #
    
    if construction==None or construction==[]:
        print("Running sdp without construction. Used block sizes are {}".format(sdp_data[0]))
        
        time.sleep(float(0.1))
        initial_sol = solve_sdp(*sdp_data)
        time.sleep(float(0.1))
        
        # Format the result and return it if floating point values are fine
        if (not exact):
            if file==None:
                return initial_sol['primal'] * mult
            return _format_optimizer_output(self, table_constructor, mult=mult, sdp_output=initial_sol, file=file)
        
        # Guess the construction in this case
        if construction==None:
            one_vector = constraints_data[-1]
            phi_vector_original = initial_sol['y']
            phi_vector_rounded, error_coeff = _round_adaptive(initial_sol['y'], one_vector)
            if error_coeff<1e-6:
                alg = FlagAlgebra(QQ, self)
                construction = alg(target_size, phi_vector_rounded)
                phipr = str(construction)
                print("The initial run gave an accurate looking construction")
                if len(phipr)<1000:
                    print("Rounded construction vector is: \n{}".format(phipr))
            else:
                print("The initial run didn't provide an accurate construction")
                construction = []
        
        # If nothing was provided or the guess failed, then round the current solution
        if construction==[]:
            rounding_output = _round_sdp_solution_no_phi(self, initial_sol, sdp_data, table_constructor, \
                                                             constraints_data, denom=denom)
            if file==None:
                return rounding_output[0] * mult
            return _format_optimizer_output(self, table_constructor, mult=mult, rounding_output=rounding_output, file=file)
    
    
    #
    # Run the optimizer (again if a construction was guessed) with the construction
    #
    
    if isinstance(construction, FlagAlgebraElement):
        phi_vectors_exact = [construction.values()]
    else:
        phi_vectors_exact = [xx.values() for xx in construction]

    #
    # Adjust the table to consider the kernel from y_rounded
    #

    print("Adjusting table with kernels from construction")
    table_constructor = self._adjust_table_phi(table_constructor, phi_vectors_exact)
    sdp_data = self._target_to_sdp_data(target_vector_exact)
    sdp_data = self._tables_to_sdp_data(table_constructor, prev_data=sdp_data)
    sdp_data = self._constraints_to_sdp_data(constraints_data, prev_data=sdp_data)
    
    #
    # Then run the optimizer
    #
    
    print("Running SDP after kernel correction. Used block sizes are {}".format(sdp_data[0]))
    time.sleep(float(0.1))
    final_sol = solve_sdp(*sdp_data)
    time.sleep(float(0.1))

    # Quickly deal with the case when no rounding is needed
    if (not exact):
        if file==None:
            return final_sol['primal'] * mult
        return _format_optimizer_output(self, table_constructor, mult=mult, sdp_output=final_sol, file=file)
    
    
    print("Starting the rounding of the result")
    rounding_output = _round_sdp_solution_phi(self, final_sol, sdp_data, table_constructor, \
                                                                     constraints_data, phi_vectors_exact, denom=denom)
    if rounding_output==None:
        print("Rounding based on construction was unsuccessful")
        rounding_output = _round_sdp_solution_no_phi(self, final_sol, sdp_data, table_constructor, \
                                                        constraints_data, denom=denom)
    
    print("Final rounded bound is {}".format(rounding_output[0]*mult))
    
    if file==None:
        return rounding_output[0] * mult
    return _format_optimizer_output(self, table_constructor, mult=mult, rounding_output=rounding_output, file=file)

In [32]:
def _flatten_matrix(mat, doubled=False):
    r"""
    Flatten a symmetric matrix, optionally double non-diagonal elements
    """
    res = []
    factor = 2 if doubled else 1
    for ii in range(len(mat)):
        res.append(mat[ii][ii])
        res += [factor*mat[ii][jj] for jj in range(ii+1, len(mat))]
    return res

def _unflatten_matrix(ls, dim, doubled=False):
    r"""
    Unflatten a symmetric matrix, optionally correct for the doubled non-diagonal elements
    """
    mat = [[0]*dim for ii in range(dim)]
    factor = 2 if doubled else 1
    index = 0
    for ii in range(dim):
        # Fill the diagonal element
        mat[ii][ii] = ls[index]
        index += 1
        # Fill the off-diagonal elements
        for jj in range(ii + 1, dim):
            mat[ii][jj] = ls[index] / factor
            mat[jj][ii] = ls[index] / factor
            index += 1
    return matrix(mat), ls[index:]

def _round(value, method=1, quotient_bound=7, denom_bound=9, denom=1024):
    r"""
    Helper function, to round a number using either 
    method=0 - simple fixed denominator
    method=1 - continued fractions
    """
    if method==0:
        return QQ(round(value*denom)/denom)
    else:
        from sage.rings.continued_fraction import continued_fraction
        cf = continued_fraction(value)
        for ii, xx in enumerate(cf.quotients()):
            if xx>=2**quotient_bound or cf.denominator(ii)>2**(denom_bound):
                if ii>0:
                    return cf.convergent(ii-1)
                return 0
        return cf.value()

def _round_list(ls, force_pos=False, method=1, quotient_bound=7, denom_bound=9, denom=1024):
    r"""
    Helper function, to round a list
    """
    if force_pos:
        return [max(_round(xx, method, quotient_bound, denom_bound, denom), 0) for xx in ls]
    else:
        return [_round(xx, method, quotient_bound, denom_bound, denom) for xx in ls]

def _round_matrix(mat, method=1, quotient_bound=7, denom_bound=9, denom=1024):
    r"""
    Helper function, to round a matrix
    """
    return matrix(QQ, [_round_list(xx, False, method, quotient_bound, denom_bound, denom) for xx in mat])

def _round_ldl(mat, method=1, quotient_bound=7, denom_bound=9, denom=1024):
    from sage.matrix.special import diagonal_matrix
    r"""
    Helper function, to round a matrix using ldl decomposition
    """
    mat_ldl = matrix(mat).block_ldlt()
    P = matrix(QQ, mat_ldl[0])
    L = matrix(QQ, _round_matrix(mat_ldl[1], method, quotient_bound, denom_bound, denom))
    D = diagonal_matrix(QQ, _round_list(mat_ldl[2].diagonal(), True, method, quotient_bound, denom_bound, denom))
    pl = P*L
    return pl*D*pl.T

def _round_adaptive(ls, onevec, denom=1024):
    r"""
    Adaptive rounding based on continued fraction and preserving an inner product
    with `onevec`
    
    If the continued fraction rounding fails fall back to a simple denominator method
    """
    best_vec = None
    best_error = 1000
    best_lcm = 1000000000
    
    orig = vector(ls)
    for resol1 in range(5, 20):
        resol2 = round(resol1*1.5)
        rls = vector([_round(xx, quotient_bound=resol1, denom_bound=resol2) for xx in orig])
        ip = rls*onevec
        if ip != 0 and abs(ip - 1)<best_error:
            if ip.as_integer_ratio()[1] > best_lcm**1.5 and ip != 1:
                continue
            best_vec = rls/ip
            best_error = abs(ip - 1)
            best_lcm = ip.as_integer_ratio()[1]
    if best_vec==None:
        rvec = vector(QQ, _round_list(ls, True, method=0, denom=denom))
        best_vec = rvec/(rvec*onevec)
    return best_vec, ((best_vec-orig)/(len(orig)**0.5)).norm()

def _fraction_print(val, thr=20):
    r"""
    Print out a fraction. If it is too big, then print an approximation
    indicating with a ? that it is not an exact value.
    """
    if len(str(val))>thr:
        return str(val.n())+"?"
    else:
        return str(val)